In [2]:
from itertools import product
import numpy as np
import itertools
from itertools import combinations
from copy import deepcopy
from operator import add,mul
from collections import defaultdict
from collections import Counter
from sage.combinat.cartesian_product import CartesianProduct_iters
import time

#Lista de los vectores de Voronoi.

def vectores_voronoi(n):
    vectors = n+1
    v=[]
    for i in range(vectors):
        v=v+[var('v{}'.format(i))]
    return (v)

#Lista de los parámetros de selling.

def parametros_selling(n):
    r=[]
    for i in range(len(vectores_voronoi(n))):
        for j in range(len(vectores_voronoi(n))):
            if i<j:
                r=r+[var('p{}{}'.format(i,j))]
    return (r)


def gram(n):
    P=matrix(SR,n) 
    for i in range(1,n+1):
        for j in range(1,n+1):
            if i==j:
                s=0                    
                for l in range(n+1):
                        if l!=i:
                            s=s+var('p{}{}'.format(min(i,l),max(i,l))) 
                P[i-1,j-1]=s
            else:
                P[i-1,j-1]=-var('p{}{}'.format(min(i,j),max(i,j)))  
                P[j-1,i-1]=P[i-1,j-1]
    return P

    
def M(n):
    """Return the Graham Matrix of an n-dimensional lattice in function of its Selling parameters"""

    M = [[0]*(n)]*(n)
    for i in range(n):
        M[i] = [-var('p%d%d' % tuple(sorted((i+1,j+1)))) for j in range(n)]
        M[i][i] = sum([var('p%d%d' % tuple(sorted((i+1,k)))) for k in range(0,n+1)]) - var('p%d%d' % (i+1,i+1))
        
    return(Matrix(M))

def d(n):
    """Return the determinant of the Graham Matrix"""
    lis=list(combinations(list(range(n+1)), 2))
    c=0
    for j in lis:
        i=list(j)
        c+=var('D%d%d' % tuple(sorted((i[0],i[1]))))*var('p%d%d' % tuple(sorted((i[0],i[1]))))
    return c/n


def G_conj(n):
    """R"""
    
    G = 0
    
    Ck = C_k(n+1)
    
    for g in Ck:
        k = Graph(g,multiedges=True).girth()
        if k == 2:
            G += mono(g)
        if k > 2:
            G += 2*(k-1)*mono(g)
    return(expand(G))


def mono(g):
    """R"""

    mono = 1
    for e in g:
        mono = mono*var('p%d%d' % (sorted(e)[0],sorted(e)[1]))
    return(mono)

def C_k(n):
    """Return all the spanning trees of the complete graph Kn"""
    
    P = prufer(n)
    
    C = []
    
    for ps in P:
        for i in range(n):
            for j in range(i+1,n):
                c = prufer_to_tree(ps)
                c.append([i,j])
                if sorted(c) not in C:
                    C.append(sorted(c))
        
    return(C)

def prufer_to_tree(ps):
    """Return the tree associated to a Prüfer sequence,
    as a list of edges"""
    
    n = len(ps)+2
    ps = list(ps)
    N = list(range(n))
    T = []
    while len(N) > 2:
        j = min(set(N)-set(ps))
        T.append(sorted([j,ps[0]]))
        N.remove(j)
        ps.remove(ps[0])
    T.append([N[0],N[1]])
    return(T)

def prufer(n):  
    """Return all Prüfer sequences of length n-2"""
    
    return(itertools.product(range(n), repeat=n-2))

In [3]:
#Numerador de G_n.
def num_G(parametros):
    N = [i for i in range(20)]
    for n in N:
         if n*(n+1) == 2*len(parametros):
            l = [parametros_selling(n)[i] == parametros[i] for i in range(len(parametros))]
            num = G_conj(n).subs(l)
    return (num)


#Denominador de G_n.
def den_G(parametros):
    N = [i for i in range(20)]
    for n in N:
         if n*(n+1) == 2*len(parametros):
            l = [parametros_selling(n)[i] == parametros[i] for i in range(len(parametros))]
            den = 12*n*(gram(n).det().subs(l))^((n+1)/n)
    return (den)


#La función que calcula la constante de cuantización de un lattice, dados sus paramátros de Selling en una lista.

def const_G(parametros):
    const = num_G(parametros)/den_G(parametros).n(30)
    return (const)

In [4]:
#Lattice A2* (A2)
const_G([1,1,1])

0.080187537

In [5]:
const_G([1/2,1/2,1/2])

0.080187537

In [6]:
const_G([0,1,1])

0.083333333

In [7]:
const_G([1,1,4])

0.089506173

In [8]:
const_G([1/4,1/4,1])

0.089506173

In [5]:
#Lattice A3*
const_G([1/4,1/4,1/4,1/4,1/4,1/4])

0.078543281

In [6]:
#Lattice A3*
const_G([1,1,1,1,1,1])

0.078543281

In [7]:
#Lattice A3
const_G([1,0,1,1,0,1])

0.078745066

In [8]:
#Lattice A4*
const_G([1,1,1,1,1,1,1,1,1,1])

0.077558757

In [9]:
#Lattice A4*
const_G([1/5,1/5,1/5,1/5,1/5,1/5,1/5,1/5,1/5,1/5])

0.077558757

In [10]:
#Lattice A4
const_G([1,0,0,1,1,0,0,1,0,1])

0.078019702

In [38]:
#Lattice A5*
const_G([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])

0.076921949

In [11]:
#Lattice A5*
const_G([1/6,1/6,1/6,1/6,1/6,1/6,1/6,1/6,1/6,1/6,1/6,1/6,1/6,1/6,1/6])

0.076921949

In [37]:
#Lattice A5
const_G([1,0,0,0,1,1,0,0,0,1,0,0,1,0,1])

0.077647458

In [20]:
G_conj(2)(1,1,1)/den_G([1,1,1]).n()

0.0801875373874480

In [26]:
G_conj(2)(1,1,1)

10

In [18]:
den_G([1,1,1])

72*sqrt(3)

In [23]:
G_conj(2)(1/3,1/3,1/3)/den_G([1/3,1/3,1/3]).n()

0.0801875373874480

In [27]:
G_conj(2)(1/3,1/3,1/3)

10/27

In [25]:
den_G([1/3,1/3,1/3])

8*sqrt(1/3)

In [33]:
M(3)

[p01 + p12 + p13            -p12            -p13]
[           -p12 p02 + p12 + p23            -p23]
[           -p13            -p23 p03 + p13 + p23]